In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep

In [2]:
url = 'https://cbr.ru/currency_base/dynamics/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

df_codes = pd.DataFrame(columns=['code', 'currency'])

for i in soup.select('option'):
    string = str(i).split()
    if 'selected=""' in string:
        row = {'code': string[2].strip('value=">'), 'currency': ' '.join(string[3:-1])}
        df_codes.loc[len(df_codes)] = row
    else:
        row = {'code': string[1].strip('value=">'), 'currency': ' '.join(string[2:-1])}
        df_codes.loc[len(df_codes)] = row

In [3]:
current_date = datetime.now().strftime('%d.%m.%Y') # today is 2023-05-11

df = pd.DataFrame(columns=['date', 'rate', 'code'])

for cur in df_codes['code']:
    url_rate = f'https://cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={cur}&UniDbQuery.From=01.01.1992&UniDbQuery.To={current_date}'
    response_rate = requests.get(url_rate)
    sleep(2)
    soup_rate = BeautifulSoup(response_rate.text, 'lxml')

    for i in soup_rate.select_one('.data'):
        string = str(i).split()
        if string != [] and all(['colspan="3">' not in string, 'class="pseudo"' not in string]):
            row = {'date': string[1].strip('<td>/'), 'rate': float(string[3].strip('<td>/').replace(',', '.')) / float(string[2].strip('<td>/').replace(',', '.')), 'code': cur}
            df.loc[len(df)] = row

In [4]:
t = df.merge(df_codes).sort_values(by=['code', 'date']).reset_index(drop=True)
t.head()

,date,rate,code,currency
0,01.01.1997,4.0000,R01010,Австралийский доллар
1,01.01.1998,3.9127,R01010,Австралийский доллар
2,01.01.1999,12.6600,R01010,Австралийский доллар
3,01.01.2000,17.6300,R01010,Австралийский доллар
4,01.01.2002,15.2946,R01010,Австралийский доллар


In [5]:
t.to_csv('current_rate.csv', index=False, sep=',')